<a href="https://colab.research.google.com/github/kunai-3txk/Prediction-of-seriously-ill-patients/blob/main/BaseLine_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
 
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
import pandas as pd

#config

In [4]:
class config:
    RANDOM_STATE=100  
    FRAC = 0.2
    N_FOLD = 5
    N_BAGGING = 3
    UNDER_SAMPLING = True
    
class paths:
    # kaggle環境ならTrue
    if 'KAGGLE_URL_BASE' in set(os.environ.keys()):
        common_path = "/kaggle/input/amp-parkinsons-disease-progression-prediction"
    
    # colaboratory環境ならTrue
    if 'COLAB_GPU' in set(os.environ.keys()):
        common_path = "/content/drive/MyDrive/ill"
    
    SUBMISSION = common_path + "/submission.csv"
    TRAIN = common_path + "/train_df.csv"
    TEST = common_path + "/test_df.csv"
    SUB_OUT = common_path + "/sub/"
     

#Load Data

In [22]:
train = pd.read_csv(paths.TRAIN)
print('train shape:',train.shape)

test = pd.read_csv(paths.TEST)
print('test shape:',test.shape)

submission = pd.read_csv(paths.SUBMISSION)
print('submission shape:',submission.shape)

train shape: (51359, 85)
test shape: (12840, 84)
submission shape: (12840, 2)


## preprocessing

In [27]:
num_cols = ['age', 'bmi', 'height', 'weight', 'icu_4', 'icu_6'
            ,'glasgow_coma_scale_1', 'glasgow_coma_scale_2','glasgow_coma_scale_3', 'glasgow_coma_scale_4'
            , 'heart_rate', 'blood_oxy', 'arterial_pressure',
            'respiratory_rate', 'temp'
            , 'blood_pressure_1', 'blood_pressure_2', 'blood_pressure_3', 'blood_pressure_4',
            'v1_heartrate_max'
            , 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9', 'v10', 'v11', 'v12', 'v13', 'v14','v15', 'v16'
            , 'w1', 'w2', 'w3', 'w4', 'w5', 'w6', 'w7', 'w8', 'w9', 'w10', 'w11', 'w12', 'w13', 'w14','w15', 'w16', 'w17', 'w18'
            , 'x1', 'x2', 'x3', 'x4', 'x5', 'x6']


X_train = train[num_cols]
X_train = X_train.fillna(-1)
y_train = train['target_label']

test_X = test[num_cols]
test_X = test_X.fillna(-1)

,age,bmi,height,weight,icu_4,icu_6,glasgow_coma_scale_1,glasgow_coma_scale_2,glasgow_coma_scale_3,glasgow_coma_scale_4,...,w15,w16,w17,w18,x1,x2,x3,x4,x5,x6
0,69.0,24.731460,175.30,76.0,25.801389,109.09,3.0,6.0,0.0,3.0,...,75.0,67.0,75.0,67.0,243.0,76.0,3.5,3.5,0.25,0.07
1,64.0,28.666129,183.00,96.0,3.639583,0.19,1.0,1.0,0.0,1.0,...,-1.0,-1.0,-1.0,-1.0,158.0,109.0,4.2,4.2,0.42,0.25
2,74.0,18.144869,166.00,50.0,0.059028,307.01,4.0,6.0,0.0,5.0,...,100.0,100.0,100.0,100.0,73.0,62.0,4.2,4.1,0.07,0.03
3,60.0,23.047667,182.90,77.1,0.014583,702.01,4.0,6.0,0.0,5.0,...,124.0,123.0,124.0,123.0,373.0,46.0,4.2,3.2,0.01,0.00
4,75.0,20.190265,160.02,51.7,0.004861,308.01,4.0,6.0,0.0,5.0,...,142.0,114.0,142.0,114.0,-1.0,-1.0,-1.0,-1.0,0.08,0.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51354,67.0,20.622289,142.20,41.7,1.352778,1409.01,4.0,6.0,0.0,5.0,...,82.0,82.0,82.0,82.0,152.0,134.0,4.4,4.1,0.02,0.00
51355,63.0,35.054523,175.20,107.6,0.005556,501.02,4.0,6.0,0.0,4.0,...,112.0,94.0,112.0,94.0,135.0,83.0,3.8,3.2,0.14,0.09
51356,73.0,24.920113,167.60,70.0,0.288194,501.06,4.0,6.0,0.0,5.0,...,77.0,64.0,77.0,64.0,201.0,101.0,3.5,3.5,0.05,0.02
51357,69.0,27.202393,170.20,78.8,0.619444,1303.01,4.0,6.0,0.0,1.0,...,185.0,158.0,185.0,158.0,208.0,113.0,3.9,3.7,0.03,0.01


In [24]:
train_X, val_X, train_y, val_y = train_test_split(X_train.values, y_train.values, test_size = 0.25, random_state=0)

train_X = torch.from_numpy(train_X).float()
train_y = torch.from_numpy(train_y).long()
val_X = torch.from_numpy(val_X).float()
val_y = torch.from_numpy(val_y).long()

train_td=TensorDataset(train_X, train_y)

train_loader = DataLoader(train_td, batch_size=15, shuffle=True)

In [36]:
class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        self.fc1 = nn.Linear(59, 128) #入力層59個（特徴量分）、中間層を128に設定
        self.fc2 = nn.Linear(128, 2)  #入力層が前の中間層の128となり、出力層は２分類ですので、2と設定
         
    def forward(self, x):
        x = F.relu(self.fc1(x)) # ReLU: max(x, 0)
        x = self.fc2(x)
        return F.log_softmax(x, dim=0)

class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        self.fc1 = nn.Linear(59, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 128)
        self.fc4 = nn.Linear(128, 128)
        self.fc5 = nn.Linear(128, 128)
        self.fc6 = nn.Linear(128, 2)
         
    def forward(self, x):
        x = F.relu(self.fc1(x)) # ReLU: max(x, 0)
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x, dim=0)

model = Net2()

In [37]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
for epoch in range(180): #学習回数500回
    total_loss = 0
     
    for train_x, train_y in train_loader:
        train_x, train_y = Variable(train_x), Variable(train_y)
        optimizer.zero_grad()
        output = model(train_x)
        loss = criterion(output, train_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() #loss.data[0]で記述するとPyTorch0.5以上ではエラーが返る
     
    if (epoch+1)%60 == 0:
        print(epoch+1, total_loss)

60 669.5211309343576
120 631.9967099055648
180 596.195278249681


In [38]:
val_X, val_y = Variable(val_X), Variable(val_y)
result = torch.max(model(val_X).data, 1)[1]
accuracy = sum(val_y.data.numpy() == result.numpy()) / len(val_y.data.numpy())
print(accuracy)

#0.9129283489096574
#0.9130841121495327

0.9130841121495327
